## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os
from tqdm.notebook import tqdm

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

## Load config

## Set up llms

In [6]:
from llama_index.llms.ollama import Ollama

llm = Ollama(
    base_url="https://535a-34-32-249-223.ngrok-free.app/",
    model='reader-lm:1.5b-fp16',
    request_timeout=200.0
)

## Extract data from web

In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
markdown_content = llm.complete(test)
print(markdown_content)

NameError: name 'test' is not defined

In [10]:
from typing import List
from pydantic import BaseModel, Field

In [11]:
DEFAULT_TAGS = ["p", "h1", "h2", "h3", "h4", "h5", "h6", "li", "b", "i", "u", "section"]
tags: List[str] = DEFAULT_TAGS

In [12]:
tags

['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li', 'b', 'i', 'u', 'section']

In [25]:
import re
import requests
import asyncio
from pathlib import Path
from tqdm.notebook import tqdm
from asyncio import Semaphore
from typing import List
from loguru import logger
from tenacity import retry, wait_random_exponential, stop_after_attempt


class CrawlData():
    DEFAULT_TAGS = ["p", "h1", "h2", "h3", "h4", "h5", "h6", "li", "b", "i", "u", "section"]
    PATTERN = r'^\d+\.\s*'
    BASE_URL = "https://www.vinmec.com"
    DATA_PATH = 'data/crawl'
    
    def __init__(self):
        self.tags: List[str] = self.DEFAULT_TAGS

        if not Path(self.DATA_PATH).exists():
            Path(self.DATA_PATH).mkdir(parents=True, exist_ok=True)

        self.data_path = self.DATA_PATH
    
    async def extract_batch(self, list_url: List[str], max_concurrency_calls: int = 10):
        from bs4 import BeautifulSoup
        
        sem = Semaphore(max_concurrency_calls)
        @retry(
            wait=wait_random_exponential(multiplier=1, min=10, max=90),
            stop=stop_after_attempt(3)
        )
        async def extract(url: str):
            phrase = {}
            current_h2 = None
            content_accumulator = []
            async with sem:
                html = requests.get(url).text
                soup = BeautifulSoup(html, 'html.parser')
                content = soup.find('div', {'class':'col-8 main-col'})
                tags = content.find_all(self.tags)
                for tag in tags:
                    tag_text = await self._extract_text_from_tag(tag)
                    if tag.name == "h2":
                        if current_h2:
                            phrase[current_h2] = " ".join(content_accumulator)
                        current_h2 = re.sub(self.PATTERN, '', tag_text.strip()) 
                        content_accumulator = []
                    else:
                        content_accumulator.append(tag_text.strip())
                if current_h2:
                    phrase[current_h2] = " ".join(content_accumulator)
            if len(phrase) != 0:
                return phrase
        coros = [extract(url) for url in tqdm(list_url, total=len(list_url))]
        phrases = await asyncio.gather(*coros)
        phrases = [phrase for phrase in phrases if phrase and phrase is not None]
        if isinstance(phrases, type(None)) or all(phrase is None for phrase in phrases):
            return
        return phrases
                
    async def _extract_text_from_tag(self, tag: str) -> str:
        from bs4 import NavigableString
        texts = []
        for element in tag.children:
            if isinstance(element, NavigableString):
                if element.strip():
                    texts.append(element.strip())
            elif element.name in self.tags:
                continue
            else:
                texts.append(element.get_text().strip())
        return "\n".join(texts)

    def get_urls(self):
        categories = {
            # 'vie/chan-thuong-chinh-hinh-y-hoc-the-thao': 287,
            # 'vie/trung-tam-cong-nghe-cao': 17,
            # 'vie/trung-tam-nhi': 990,
            # 'vie/trung-tam-suc-khoe-phu-nu': 902,
            # 'vie/trung-tam-vu': 32,
            # 'vie/suc-khoe-tong-quat': 1167,
            # 'vie/than-kinh': 7,
            # 'vie/tieu-hoa-gan-mat': 414,
            # 'vie/tim-mach': 290,
            # 'vie/ung-buou': 235,
            # 'vie/trung-tam-vac-xin': 230,
            # 'vie/te-bao-goc-va-cong-nghe-gen': 21,
            # 'vie/y-hoc-co-truyen': 97,
            # 'vie/tham-my': 86,
            # 'vie/dinh-duong': 226,
            # 'vie/song-khoe': 153,
            'vie/mien-dich-di-ung': 16
        }
        for category, pages in categories.items():
            print(category, pages)
            for page in range(1, pages+1):
                yield f"{self.BASE_URL}/{category}/page_{page}"

    async def get_article_urls(self, page_url: str) -> List[str]:
        from bs4 import BeautifulSoup
        
        html = requests.get(page_url).text
        articles = BeautifulSoup(html, 'html.parser')
        tags = articles.find_all('div', {'class': 'col-3'})
        article_urls = [self.BASE_URL + tag.find('a').get('href') for tag in tags]
        
        return article_urls

    def _save_json(self, data, page_url):
        logger.info(f"data {data}")
        if data is not None and len(data) != 0:
            category_folder = page_url.split('/')[-2]
            if not Path(self.data_path + '/' + category_folder).exists():
                Path(self.data_path + '/' + category_folder).mkdir(exist_ok=True, parents=True)
            
            page_id = page_url.split('/')[-1]
            logger.info(f'Save {category_folder + '/' + page_id} into : {self.data_path}')
            with open(self.data_path + f'/{category_folder}/{page_id}.json', 'w') as f:
                json.dump(data, f, indent=4, ensure_ascii=False)
            
    async def run(self):
        page_urls = self.get_urls()
        
        for page_url in page_urls:
            article_urls = await self.get_article_urls(page_url)
            logger.info(f"Extract {len(article_urls)} article from {page_url}")
            articles = await self.extract_batch(article_urls)
            
            self._save_json(articles, page_url)
            

In [26]:
crawl = CrawlData()

In [27]:
res = await crawl.run()

vie/mien-dich-di-ung 16


2024-10-14 17:31:35.315 | INFO     | __main__:run:128 - Extract 8 article from https://www.vinmec.com/vie/mien-dich-di-ung/page_1


  0%|          | 0/8 [00:00<?, ?it/s]

2024-10-14 17:31:36.592 | INFO     | __main__:_save_json:112 - data [{'Triệu chứng khi bị dị ứng lông chó': 'Người bị\ndị ứng lông chó\nthường có các triệu chứng tương tự như các loại dị ứng mũi khác, bao gồm: Ho, khò khè, khó thở. Đỏ mắt, ngứa mắt và chảy nước mắt. Chảy nước mũi, hắt hơi, ngứa mũi và\nnghẹt mũi\n. Một số người bị dị ứng lông chó không chỉ cảm thấy ngứa ngáy và hắt hơi mà còn có thể xảy ra phản ứng trên da. Da của bệnh nhân có thể nổi mẩn đỏ sau khi bị chó liếm. Đặc biệt nguy hiểm, những người mắc\nhen suyễn\nhoặc\ndị ứng với thú cưng\ncó thể gặp phải các triệu chứng nghiêm trọng.', 'Các nguyên nhân gây ra dị ứng lông chó': 'Các protein gây dị ứng có mặt trong da chết, lông, nước bọt và nước tiểu của chó.\nHệ miễn dịch\nphản ứng quá mức với những protein này, gây ra phản ứng dị ứng. Mức độ dị ứng còn phụ thuộc vào giống chó do sự đa dạng về loại dị nguyên và loại lông. Ngoài ra, chất gây dị ứng từ chó thường bám vào lông, thảm, quần áo, tường, đệm ghế và có thể tồn tại

  0%|          | 0/8 [00:00<?, ?it/s]

...

2024-10-14 17:31:59.179 | INFO     | __main__:_save_json:112 - data [{'Viêm da tiếp xúc dị ứng là gì?': 'Viêm da tiếp xúc\nlà một tình trạng phổ biến trong các vấn đề da liễu, xuất hiện khi da bị dị ứng khi tiếp xúc trực tiếp với các chất kích thích hoặc gây dị ứng. Các triệu chứng của viêm da tiếp xúc thường bao gồm cảm giác ngứa ngáy và không thoải mái, đồng thời cũng ảnh hưởng tiêu cực đến vẻ đẹp tổng thể của làn da.', 'Vì sao dễ mắc bệnh viêm da tiếp xúc?': 'Các nguyên nhân gây\nviêm da tiếp xúc\nbao gồm: Tiếp xúc với sản phẩm mới:\nViêm da tiếp xúc có thể xuất hiện sau khi tiếp xúc với sản phẩm mới sau vài giờ hoặc vài ngày đến vài tuần. Tiếp xúc với các chất gây dị ứng:\nĐeo trang sức chứa niken, sử dụng nước hoa, mỹ phẩm, sơn móng tay, giày dép, các sản phẩm từ cao su, chứa các thành phần hóa học có thể gây ra bệnh lý này. Sử dụng thuốc:\nMột số loại thuốc như thuốc bôi chứa kháng sinh,\nbenzocaine\nvà thimerosal cũng có thể dẫn đến viêm da tiếp xúc. Tiếp xúc với các loại cây:\n

  0%|          | 0/8 [00:00<?, ?it/s]

2024-10-14 17:32:01.290 | INFO     | __main__:_save_json:112 - data [{'Bạch cầu hạt': '1.1 Đặc điểm của bạch cầu hạt đặc trưng bởi các hạt nhuộm màu khác nhau trong tế bào chất được quan sát dưới kính hiển vi quang học, có các đặc điểm sau: Bạch cầu hạt Có khả năng thay đổi hình dạng, xuyên mạch qua vách giữa các tế bào và chuyển động có hướng bằng chân giả (giống amip) với tốc độ 40 mm/phút về phía bị viêm. Có khả năng thực bào, ẩm bào. Có khả năng đáp ứng với những kích thích hóa học như một số chất do mô bị viêm sản xuất hoặc do vi khuẩn tạo ra, hoặc những chất hoá học được đưa từ ngoài vào cơ thể. Hoá ứng động dương tính tức là thu hút, tập trung bạch cầu tới chỗ viêm, còn hoá ứng động âm tính tức là xua đuổi bạch cầu ra xa hơn. Có khả năng đáp ứng với những kích thích nhiệt học. 1.2 Phân loại bạch cầu hạt Có ba loại\n, được đặt tên theo các thuộc tính nhuộm màu của từng loại hạt, đó là: bạch cầu hạt Bạch cầu đa nhân trung tính (neutrophil) Bạch cầu ái kiềm (basophil) Bạch cầu ái t

  0%|          | 0/5 [00:00<?, ?it/s]

2024-10-14 17:32:02.900 | INFO     | __main__:_save_json:112 - data [{'Dị ứng ở trẻ em là gì?': 'Dị ứng ở trẻ em\nlà tình trạng mà cơ thể trẻ phản ứng một cách quá mức với các dị nguyên lạ thông qua hệ thống miễn dịch. Các bệnh dị ứng hay gặp ở những trẻ có người thân cũng mắc dị ứng. Dị ứng ở trẻ em không chỉ ảnh hưởng đến các hoạt động hàng ngày và giấc ngủ, mà trong một số trường hợp nghiêm trọng còn đe dọa đến tính mạng của trẻ nếu không được phát hiện và điều trị kịp thời.', 'Các bệnh dị ứng hay gặp ở trẻ': '2.1 Viêm da cơ địa (Chàm thể tạng) Viêm da cơ địa\nlà tình trạng\ndị ứng hay gặp ở trẻ\n, đặc trưng bởi sự xuất hiện của các mụn nước nhỏ trên các vùng da đỏ, thường gặp ở mặt và cánh tay hoặc thậm chí rải rác khắp cơ thể. Những mụn nước này không chỉ gây cảm giác ngứa rát mà còn tạo điều kiện cho vi khuẩn xâm nhập và gây nhiễm khuẩn khi chúng vỡ chảy dịch. Việc chẩn đoán bệnh này đôi khi khó khăn do dễ bị nhầm lẫn với các bệnh lý về da khác, chính vì thế việc khám chuyên khoa

## Glob all sentence into huggingface dataset

In [6]:
import glob
import random as rd
from datasets import load_dataset

In [15]:
folder = glob.glob('data/crawl/*/*')
len(folder)

3214

In [16]:
file_names = glob.glob('data/crawl/*/*')
len(file_names), file_names[0]

(3214, 'data/crawl/tham-my/page_36.json')

In [56]:
test = []
with open('data/crawl/tham-my/page_36.json', 'r') as f:
    articles = json.load(f)
    for article in articles:
        for key, value in article.items():
            if key.startswith("Trắc nghiệm:"):
                continue
            sentence = {
                "id": rd.randint(1, 1000000),
                "sentence1": key,
                "sentence2": value
            }
            test.append(sentence)

In [58]:
len(test)

25

In [60]:
data = []
for file_name in tqdm(file_names, total=len(file_names)):
    with open(file_name, 'r') as file:
        articles = json.load(file)
        for article in articles:
            for key, value in article.items():
                if key.startswith('Trắc nghiệm:'):
                    continue
                sentence = {
                    "id": rd.randint(1, 1000000),
                    "sentence1": key,
                    "sentence2": value
                }
                data.append(sentence)
            

  0%|          | 0/3214 [00:00<?, ?it/s]

In [61]:
len(data)

89387

In [62]:
with open('test.json', 'w') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [63]:
ds = load_dataset('json', data_files='test.json', split='all')

Generating train split: 0 examples [00:00, ? examples/s]

In [64]:
ds = ds.train_test_split(test_size=0.1)

In [65]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2'],
        num_rows: 80448
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2'],
        num_rows: 8939
    })
})

In [66]:
ds.push_to_hub('BookingCare/ViHealth-NLI')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/81 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BookingCare/ViHealth-NLI/commit/011235612dd1397761c4e5cf3a41f0932538b70f', commit_message='Upload dataset', commit_description='', oid='011235612dd1397761c4e5cf3a41f0932538b70f', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
ds['train'][0]

{'id': 897497,
 'sentence1': 'Hút mỡ bụng có hại không?',
 'semtence2': ' Nếu người thực hiện đáp ứng đủ những yêu cầu sức khỏe để\nthì phương pháp này hoàn toàn vô hại. Tuy nhiên, một số trường hợp sau phẫu thuật có thể gặp phải một số biến chứng, trong đó phổ biến nhất là do quá liều thuốc gây tê. Vì vậy, việc gây tê trong hút mỡ bụng là cực kỳ quan trọng, không được để thuốc vào cơ thể quá nhanh sẽ gây ra tình trạng quá liều và cần phải cấp cứu kịp thời, nếu không sẽ ảnh hưởng đến tính mạng của người bệnh. Một số trường hợp nguy kịch thường có biểu hiện co giật hay trụy mạch. Vì vậy, khi thực hiện tiêm thuốc gây tê cho người thực hiện cần tiêm một cách từ từ và quan sát biểu hiện lâm sàng của bệnh nhân xem có khó chịu hay bất thường hay không, nếu có thì cần ngừng tiêm loại thuốc đó ngay. hút mỡ bụng là phương pháp làm đẹp tương đối an toàn và ít gây đau đớn cho người bệnh. Để tối ưu hóa những điểm tích cực của\nthì người bệnh cần thực hiện ở những cơ sở thẩm mỹ có uy tín, chất lượn